## DIA 073: Creación de una API REST con Flask y SQLAlchemy

Hoy aprenderemos a crear una API REST con Flask y SQLAlchemy, que nos permitirá manejar datos de manera eficiente en una base de datos.

✅ ¿Qué aprenderemos?

📌 Configurar Flask como backend.
📌 Usar SQLAlchemy como ORM para manejar datos.
📌 Crear una API REST con endpoints CRUD (Create, Read, Update, Delete).
✅ Ejemplo práctico:
Una API de gestión de productos con endpoints para crear, listar, actualizar y eliminar productos.

🛠️ 1. Instalación de Paquetes Necesarios
Ejecutar en la terminal:

sh
Copiar
Editar
pip install flask flask_sqlalchemy flask_marshmallow marshmallow-sqlalchemy
📌 Explicación de los paquetes:

flask: Framework principal para la API.
flask_sqlalchemy: ORM para manejar bases de datos.
flask_marshmallow: Para serializar y deserializar datos.
🖥️ 2. Código Completo (api_flask.py)
python
Copiar
Editar
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from flask_marshmallow import Marshmallow

# ---------------------------
# 1. Configuración Inicial
# ---------------------------
app = Flask(__name__)
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///productos.db"
app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False

db = SQLAlchemy(app)
ma = Marshmallow(app)

# ---------------------------
# 2. Definir el Modelo de Producto
# ---------------------------
class Producto(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    nombre = db.Column(db.String(100), unique=True, nullable=False)
    precio = db.Column(db.Float, nullable=False)

    def __init__(self, nombre, precio):
        self.nombre = nombre
        self.precio = precio

# ---------------------------
# 3. Serialización con Marshmallow
# ---------------------------
class ProductoSchema(ma.SQLAlchemyAutoSchema):
    class Meta:
        model = Producto

producto_schema = ProductoSchema()
productos_schema = ProductoSchema(many=True)

# ---------------------------
# 4. Crear la Base de Datos
# ---------------------------
with app.app_context():
    db.create_all()

# ---------------------------
# 5. Endpoints de la API
# ---------------------------

# a. Crear un Producto
@app.route("/producto", methods=["POST"])
def crear_producto():
    nombre = request.json["nombre"]
    precio = request.json["precio"]

    nuevo_producto = Producto(nombre, precio)
    db.session.add(nuevo_producto)
    db.session.commit()

    return producto_schema.jsonify(nuevo_producto)

# b. Obtener todos los Productos
@app.route("/productos", methods=["GET"])
def obtener_productos():
    productos = Producto.query.all()
    return productos_schema.jsonify(productos)

# c. Obtener un Producto por ID
@app.route("/producto/<int:id>", methods=["GET"])
def obtener_producto(id):
    producto = Producto.query.get(id)
    if producto:
        return producto_schema.jsonify(producto)
    return jsonify({"mensaje": "❌ Producto no encontrado"}), 404

# d. Actualizar un Producto
@app.route("/producto/<int:id>", methods=["PUT"])
def actualizar_producto(id):
    producto = Producto.query.get(id)
    if producto:
        producto.nombre = request.json["nombre"]
        producto.precio = request.json["precio"]
        db.session.commit()
        return producto_schema.jsonify(producto)
    return jsonify({"mensaje": "❌ Producto no encontrado"}), 404

# e. Eliminar un Producto
@app.route("/producto/<int:id>", methods=["DELETE"])
def eliminar_producto(id):
    producto = Producto.query.get(id)
    if producto:
        db.session.delete(producto)
        db.session.commit()
        return jsonify({"mensaje": "✅ Producto eliminado"})
    return jsonify({"mensaje": "❌ Producto no encontrado"}), 404

# ---------------------------
# 6. Ejecutar el Servidor
# ---------------------------
if __name__ == "__main__":
    app.run(debug=True)
🚀 Cómo Ejecutar la API
1️⃣ Ejecutar en la terminal:

sh
Copiar
Editar
python api_flask.py
2️⃣ Abrir en el navegador o Postman:
📌 http://127.0.0.1:5000/productos

📌 Prueba los Endpoints con curl

sh
Copiar
Editar
# Crear un producto
curl -X POST http://127.0.0.1:5000/producto -H "Content-Type: application/json" -d '{"nombre": "Laptop", "precio": 799.99}'

# Obtener todos los productos
curl -X GET http://127.0.0.1:5000/productos

# Obtener un producto por ID
curl -X GET http://127.0.0.1:5000/producto/1

# Actualizar un producto
curl -X PUT http://127.0.0.1:5000/producto/1 -H "Content-Type: application/json" -d '{"nombre": "Laptop Gamer", "precio": 999.99}'

# Eliminar un producto
curl -X DELETE http://127.0.0.1:5000/producto/1
🔍 Explicación de las Principales Implementaciones
🔹 📂 Configuración de Flask y SQLAlchemy

SQLAlchemy(app) conecta Flask con SQLite.
db.create_all() crea la base de datos si no existe.
🔹 📊 Definición del Modelo de Datos

class Producto(db.Model) define los atributos nombre y precio.
🔹 🔄 Serialización con Marshmallow

ProductoSchema(ma.SQLAlchemyAutoSchema) convierte modelos a JSON.
🔹 📡 CRUD Completo con Flask

POST /producto → Agregar un nuevo producto.
GET /productos → Listar todos los productos.
GET /producto/{id} → Obtener un producto específico.
PUT /producto/{id} → Actualizar un producto.
DELETE /producto/{id} → Eliminar un producto.